In [ ]:
!pip install sgp4 pandas numpy matplotlib scikit-learn requests


In [ ]:
import os
import requests

# Create data folder
os.makedirs("data", exist_ok=True)

# Download the TLE file from Celestrak
url = "https://celestrak.com/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle"
response = requests.get(url)
tle_path = "data/starlink.txt"
with open(tle_path, "w") as f:
    f.write(response.text)

print("TLE data downloaded to", tle_path)


In [ ]:
from sgp4.api import Satrec
import pandas as pd

def parse_tle_file(filename):
    with open(filename, "r") as f:
        lines = f.readlines()
    sats = []
    for i in range(0, len(lines) - 2, 3):
        name = lines[i].strip()
        l1 = lines[i + 1].strip()
        l2 = lines[i + 2].strip()
        try:
            sat = Satrec.twoline2rv(l1, l2)
            sats.append({
                "name": name,
                "epoch": sat.jdsatepoch,
                "inclo": sat.inclo,
                "eccentricity": sat.ecco,
                "raan": sat.nodeo,
                "argpo": sat.argpo,
                "mean_anomaly": sat.mo,
                "mean_motion": sat.no_kozai
            })
        except:
            continue
    return pd.DataFrame(sats)

df = parse_tle_file("data/starlink.txt")
df.head()


In [ ]:
from sklearn.preprocessing import StandardScaler

# Sort by time
df = df.sort_values("epoch")

# Select orbital parameters
features = ["inclo", "eccentricity", "raan", "argpo", "mean_anomaly", "mean_motion"]

# Scale the data
scaler = StandardScaler()
X = scaler.fit_transform(df[features])
y = df[features].shift(-1).dropna().values
X = X[:-1]

print("X shape:", X.shape)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X, y)
preds = model.predict(X)

mse = mean_squared_error(y, preds)
print(f"Mean Squared Error: {mse:.6f}")


In [ ]:
import matplotlib.pyplot as plt

actual = y
predicted = preds

fig, axs = plt.subplots(2, 3, figsize=(14, 8))
labels = ["Inclination", "Eccentricity", "RAAN", "Arg Perigee", "Mean Anomaly", "Mean Motion"]

for i in range(6):
    row, col = i // 3, i % 3
    axs[row, col].plot(actual[:, i], label="Actual")
    axs[row, col].plot(predicted[:, i], label="Predicted", linestyle="--")
    axs[row, col].set_title(labels[i])
    axs[row, col].legend()

plt.tight_layout()
plt.show()


In [ ]:
# 🛰️ Starlink Orbit Prediction with Machine Learning

This notebook:
- Downloads live Starlink TLE data from [Celestrak](https://celestrak.com/)
- Parses orbital elements using the SGP4 library
- Trains a Linear Regression model to predict next-orbit parameters
- Visualizes predicted vs actual results

Built by Stephen Baird II  
🚀 ERAU | Machine Learning in Aerospace Engineering  
